# Regresion logistica polinomica
Autor: Jose Monzon

Datos brindados para el laboratorio por el profesor. Tomados de Kaggle. 
Para este trabajo se utilizaron muchos de los ejercicios vistos en la clase de Samuel Chavez, 2023. Repositorio: https://github.com/samuelchvez/python-ml-2021


## Task 1
Leer el archivo `.csv` a un p array.

In [35]:
import numpy as np
import csv
from matplotlib import pyplot as plt

raw_data = csv.reader(open('framingham.csv', 'r'))
raw_data = np.array(list(raw_data))

In [36]:
headers = list(raw_data[0])
data = raw_data[1:]

y = data[:,headers.index('TenYearCHD')]
y = y.reshape(-1,1)
y = y.astype(int)

# Variables to be used in the model: age, sysBP, diaBP, prevalentHyp

age = data[:,headers.index('age')].reshape(-1,1).astype(float)
sysBP = data[:,headers.index('sysBP')].reshape(-1,1).astype(float)
diaBP = data[:,headers.index('diaBP')].reshape(-1,1).astype(float)
prevalentHyp = data[:,headers.index('prevalentHyp')].reshape(-1,1).astype(int)

X = np.concatenate((age, sysBP, diaBP, prevalentHyp), axis=1)
X


array([[ 39. , 106. ,  70. ,   0. ],
       [ 46. , 121. ,  81. ,   0. ],
       [ 48. , 127.5,  80. ,   0. ],
       ...,
       [ 48. , 131. ,  72. ,   0. ],
       [ 44. , 126.5,  87. ,   0. ],
       [ 52. , 133.5,  83. ,   0. ]])

# Task 2
Ajustar un modelo logístico polinomial en base al juego de datos cargado de forma matricial que relaciona las
variables independientes que usted considere apropiadas (puede no utilizar todas las componentes de X), con la
variable dependiente de salida (sufre o no sufre un paro cardíaco).

In [40]:
def sigmoid(x): return 1 / (1 + np.exp(-x))

def cost(X, y, t):
    return np.sum((np.dot(X, t) - y) ** 2) / len(X)

def logistic_regression(X, y, l, iterations):
    theta = np.zeros((X.shape[1], 1))
    m = len(X)
    for i in range(iterations):
        z = np.dot(X, theta)
        h = sigmoid(z)
        gradient = np.dot(X.T, (h - y)) / m
        theta -= l * gradient
    return theta

def predict(X, theta):
    return (sigmoid(X @ theta) >= 0.5).astype(int)

## Task 3
Utilice la implementación vectorial del algoritmo de regresión logística (descenso del gradiente visto en clase)

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

w = logistic_regression(X, y, 0.0001, 10000)

y_hat = predict(X, w)

print('Accuracy: ', np.sum(y_hat == y) / len(y))

Accuracy:  0.8489853704577631


## Task 4
Usando cross-validation determine el grado del polinomio que mejor describe la nube de puntos (encuentre el mejor balance entre apego a los datos de entrenamiento y generalización para datos previamente no observados).

In [42]:
def training_test(data, train_size = 0.8):
    np.random.shuffle(data)
    train_size = int(train_size * len(data))
    return data[:train_size], data[train_size:]

def mean_squared_error(X, y, degree):
    X_poly = np.column_stack([X ** i for i in range(1, degree + 1)])
    X_poly = np.insert(X_poly, 0, 1, axis = 1)
    theta = np.linalg.inv(X_poly.T.dot(X_poly)).dot(X_poly.T).dot(y)
    y_pred = np.dot(X_poly, theta)
    mse = np.sum((y - y_pred) ** 2) / len(X)
    return mse

def cross_validation(data, degree, k = 3):
    np.random.shuffle(data)
    mse = 0
    for i in range(k):
        test_i = np.arange(i, len(data), k)
        train_i = np.setdiff1d(np.arange(len(data)), test_i)
        mse += mean_squared_error(data[train_i][:, 0], data[train_i][:, 1], degree)
    mse /= k
    return mse
        
def best_degree(data, max):
    mse_best = float('inf')
    for d in range(1, max):
        mse = cross_validation(data, d)
        if mse < mse_best:
            mse_best = mse
            degree_best = d
    return degree_best
    
data_train, data_test = training_test(np.column_stack((X, y)), train_size = 0.8)

best_degree = best_degree(data_train, 50) # Probamos con distintos grados de polinomio para ver cual es el mejor

print("best_degree = ", best_degree)

best_degree =  5


## Task 5 : Analisis de resultados

In [43]:
print("Accuracy: ", np.sum(y_hat == y) / len(y))
print("Best Degree: ", best_degree)

Accuracy:  0.8489853704577631
Best Degree:  5


Utlizando las variables de edad, presion sistolica, presion diastolica y si el paciente era hipertenso, se obtuvo un modelo con un 84,8% de presicion al realizarle una regresion logistica y comprobarlo utilizando validacion cruzada. Asimismo se obtuvo que el grado optimo del polinomio es cuando este cuenta con 5 grados.

Este modelo puede ser utilizado para predecir si un paciente sufre un paro cardiaco o no, en base a su flujo sanguineo, presion arterial y edad. Sin embargo al no tener una accuracy lo suficientemente alta como para ser utilizado en la practica, se recomienda realizar mas pruebas para mejorar el modelo.